In [ ]:
# En este notebook se hizo una restructuración de la entrega previa. 
# Se procedió a hacer una limpia de las variables relevantes para las preguntas. 
# Sin embargo, todavía no se construyen las consultas que responden las preguntas. 
# Esperamos que mañana tengamos la respuesta a todas las preguntas de la etapa 1-3
# Podemos observar que algunas preguntas más generales se pueden responder, como: 
# ¿Cuáles son las distintas rutas en todos los años?
# ¿Cuáles son los productos de todos los años? 

In [1]:
import pandas as pd
import numpy as np
import sys
import glob
import os.path
import re

In [2]:
folder_name = "./data/"

data_path = os.path.join(folder_name,'*xlsx')

files = glob.glob(data_path)

In [3]:
files

['./data/v2017.xlsx',
 './data/v2012.xlsx',
 './data/v2013.xlsx',
 './data/v2015.xlsx',
 './data/v2018.xlsx',
 './data/v2014.xlsx',
 './data/v2016.xlsx']

In [4]:
years = []
for file in files:
    df = pd.read_excel(file, sheet_name = None)
    year = file.replace('./data/v', '')
    year = year.replace('.xlsx', '')
    years.append([df, year])
    df['Hoja1'].to_pickle(year)
    

In [7]:
def extract_product_tables(df): 
    cols = ['PRODUCTO', 'CARGA', 'EXISTENCIA', 'DEVOLUCION', 'VENTA TOTAL', 'PRECIO * PRODUC', '$', 'VENTA EN LT', 'RUTA', 'DIA', 'SEMANA']
    productos = pd.DataFrame(columns= cols)
    test_copy = df
    for i in range(int(df.shape[0])):
        for j in range(int(df.shape[1])):
            if type(df.iloc[i, j]) == unicode and ('RUTA:' in df.iloc[i, j] or 'IVAN' in df.iloc[i, j]):
                ruta = ''
                dia = ''
                semana = ''
                ruta = df.iloc[i, j]
                dia = df.iloc[i - 1, j]
                semana = df.iloc[i - 2, j]
                new_entry = test_copy.iloc[i + 2:i + 10, j:j + 12].T
                new_entry['RUTA'] = ruta
                new_entry['DIA'] = dia
                new_entry['SEMANA'] = semana
                new_entry.columns = cols
                if ruta != '' and not 'QUESO' in ruta:
                    productos = productos.append(new_entry, ignore_index=True)
    return productos

In [8]:
cols = ['PRODUCTO', 'CARGA', 'EXISTENCIA', 'DEVOLUCION', 'VENTA TOTAL', 'PRECIO * PRODUC', '$', 'VENTA EN LT', 'RUTA', 'DIA', 'SEMANA', 'ANIO']
productos = pd.DataFrame(columns= cols)
for year in years:
    year_raw = year[0]['Hoja1']
    year_name = year[1]
    year_products = extract_product_tables(year_raw)
    year_products['ANIO'] = year_name
    productos = productos.append(year_products, ignore_index=True)

In [9]:
productos.shape

(90919, 12)

In [10]:
productos.to_pickle('productos')

In [50]:
productos = pd.read_pickle('productos')

In [29]:
# LIMPIAA!

In [51]:
# Queremos eliminar todas las observaciones que no son rutas
# Maldito Ivan.
rutas = []
for ruta in productos['RUTA']:
    if not ruta in rutas:
        rutas.append(ruta)
rutas

[u'RUTA: FERMIN',
 u'RUTA: JUGUERAS',
 u'RUTA: SAN MARCOS',
 u'RUTA:MARTIN',
 u'RUTA: MARTIN',
 u'RUTA: ERICK',
 u'RUTA: JESUS',
 u'RUTA: TRADICION',
 u'RUTA IVAN',
 u'QHR IVAN',
 u'QH IVAN',
 u'RUTA IVAN ',
 u'BALANCE SEMANAL IVAN ',
 u'IVAN ']

In [52]:
productos = productos.loc[productos['RUTA'] != 'QHR IVAN']
productos = productos.loc[productos['RUTA'] != 'QH IVAN']
productos = productos.loc[productos['RUTA'] != 'BALANCE SEMANAL IVAN ']
productos = productos.loc[productos['RUTA'] != 'IVAN ']
rutas = []
for ruta in productos['RUTA']:
    if not ruta in rutas:
        rutas.append(ruta)
rutas

[u'RUTA: FERMIN',
 u'RUTA: JUGUERAS',
 u'RUTA: SAN MARCOS',
 u'RUTA:MARTIN',
 u'RUTA: MARTIN',
 u'RUTA: ERICK',
 u'RUTA: JESUS',
 u'RUTA: TRADICION',
 u'RUTA IVAN',
 u'RUTA IVAN ']

In [53]:
# Ahora redefinimos repetidos: 
productos = productos.replace('RUTA:MARTIN', 'RUTA: MARTIN')
productos = productos.replace('RUTA IVAN ', 'RUTA: IVAN')
productos = productos.replace('RUTA IVAN', 'RUTA: IVAN')
rutas = []
for ruta in productos['RUTA']:
    if not ruta in rutas:
        rutas.append(ruta)
rutas

[u'RUTA: FERMIN',
 u'RUTA: JUGUERAS',
 u'RUTA: SAN MARCOS',
 'RUTA: MARTIN',
 u'RUTA: ERICK',
 u'RUTA: JESUS',
 u'RUTA: TRADICION',
 'RUTA: IVAN']

In [54]:
productos_nombres = []
for producto in productos['PRODUCTO']:
    if not producto in productos_nombres:
        productos_nombres.append(producto)
productos_nombres

[u'PRODUCTO8',
 u'PRODUCTO3',
 u'PRODUCTO7',
 u'PRODUCTO2',
 u'PRODUCTO1',
 u'PRODUCTO5',
 u'PRODUCTO9',
 nan,
 nan,
 nan,
 nan,
 u'PRODUCTO',
 u'PRODUCTO10',
 u'Queso Hebra',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 u'Queso fresco',
 u'Queso a\xf1ejo',
 u'Queso panela',
 u'PRODUCTO4',
 u'Requeson',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 u'PRODUCTO1 bolsa',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan

In [55]:
# Elmininamos nan's en productos
productos = productos[productos['PRODUCTO'].isnull() == False]
productos_nombres = []
for producto in productos['PRODUCTO']:
    if not producto in productos_nombres:
        productos_nombres.append(producto)
productos_nombres

[u'PRODUCTO8',
 u'PRODUCTO3',
 u'PRODUCTO7',
 u'PRODUCTO2',
 u'PRODUCTO1',
 u'PRODUCTO5',
 u'PRODUCTO9',
 u'PRODUCTO',
 u'PRODUCTO10',
 u'Queso Hebra',
 u'Queso fresco',
 u'Queso a\xf1ejo',
 u'Queso panela',
 u'PRODUCTO4',
 u'Requeson',
 u'PRODUCTO1 bolsa',
 6,
 8,
 u'Queso hebra',
 u'LTRS',
 u'TOTAL QUESO',
 50,
 10.6,
 55.3,
 12.5,
 41.629999999999995,
 45.78,
 44.25,
 45.05,
 0,
 49.8,
 57.6,
 11.95,
 40,
 77.1,
 21.6,
 89.7,
 15,
 49.55,
 49.08,
 u'MS',
 67.8,
 52.8,
 25,
 74.5,
 u'TOTAL QUES',
 5.3,
 58.5,
 13.3,
 7.5,
 73.5,
 10.8,
 52.5,
 8.5,
 11.6,
 42.7,
 2.7,
 14.9,
 75,
 4.4,
 15.8,
 44.9,
 2.8,
 59.5,
 2.6,
 5,
 57.4,
 41.5,
 4.6,
 42.3,
 29.4,
 16.6,
 62.4,
 2.5,
 45,
 8.7,
 u'L-G-I',
 0.3,
 41.6,
 37.5,
 36,
 39.5,
 30,
 43.7,
 5.4,
 50.1,
 27.9,
 43,
 54.5,
 21,
 58.92,
 22.5,
 70.9,
 34.8,
 45.3,
 54.9,
 58.099999999999994,
 4,
 0.4,
 49.6,
 52,
 3.5,
 50.5,
 2,
 43.2,
 56.8,
 46.05,
 89.2,
 62.7,
 61,
 48.78,
 26.400000000000002,
 51.900000000000006,
 1.96,
 60,
 25.1

In [56]:
# Eliminamos productos invalidos
productos['PRODUCTO'] = productos['PRODUCTO'].apply(unicode)
productos = productos[productos['PRODUCTO'].str.contains("PRO") | productos['PRODUCTO'].str.contains("QUES") | productos['PRODUCTO'].str.contains("Ques")]
productos_nombres = []
for producto in productos['PRODUCTO']:
    if not producto in productos_nombres:
        productos_nombres.append(producto)
productos_nombres

[u'PRODUCTO8',
 u'PRODUCTO3',
 u'PRODUCTO7',
 u'PRODUCTO2',
 u'PRODUCTO1',
 u'PRODUCTO5',
 u'PRODUCTO9',
 u'PRODUCTO',
 u'PRODUCTO10',
 u'Queso Hebra',
 u'Queso fresco',
 u'Queso a\xf1ejo',
 u'Queso panela',
 u'PRODUCTO4',
 u'PRODUCTO1 bolsa',
 u'Queso hebra',
 u'TOTAL QUESO',
 u'TOTAL QUES']

In [57]:
# Estandarizamos nombres
productos = productos.replace('PRODUCTO1 bolsa', 'PRODUCTO1')
productos = productos.replace('TOTAL QUES', 'TOTAL QUESO')
productos_nombres = []
for producto in productos['PRODUCTO']:
    if not producto in productos_nombres:
        productos_nombres.append(producto)
productos_nombres

[u'PRODUCTO8',
 u'PRODUCTO3',
 u'PRODUCTO7',
 u'PRODUCTO2',
 u'PRODUCTO1',
 u'PRODUCTO5',
 u'PRODUCTO9',
 u'PRODUCTO',
 u'PRODUCTO10',
 u'Queso Hebra',
 u'Queso fresco',
 u'Queso a\xf1ejo',
 u'Queso panela',
 u'PRODUCTO4',
 u'Queso hebra',
 u'TOTAL QUESO']

In [58]:
# Eliminamos todas las observaciones que sean en realidad los nombres de las columnas
productos = productos.loc[productos['CARGA'] != 'CARGA']

In [59]:
dias = []
for dia in productos['DIA']:
    if not dia in dias:
        dias.append(dia)
dias

[nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 u'SABADO',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 u'DIA:LUNES ',
 u'DIA LUNES ',
 u'DIA DOMIN',
 u'DIA MARTES ',
 u'DIA MERTES ',
 u'MIERCOLES',
 u'DIA MIERCOLES ',
 u'DIA MIERCO',
 u'jueves',
 u'DIA JUEVES  ',
 u'DIA JUEVES ',
 u' DIA VIERNES ',
 nan,
 u'DIA VIERNES ',
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan,
 nan

In [60]:
# Filtramos aquellas observaciones que claramente no son dias
productos_dias_nan = productos[productos['DIA'].isnull() == True]
productos = productos[productos['DIA'].str.contains("SAB") | productos['DIA'].str.contains("DOM") | 
                         productos['DIA'].str.contains("LUN") | productos['DIA'].str.contains("MAR") | 
                         productos['DIA'].str.contains("MIER") | productos['DIA'].str.contains("JUE") | 
                         productos['DIA'].str.contains("VIE") | productos['DIA'].str.contains("sab") | 
                         productos['DIA'].str.contains("dom") | productos['DIA'].str.contains("lun") | 
                         productos['DIA'].str.contains("mar") | productos['DIA'].str.contains("mier") | 
                         productos['DIA'].str.contains("jue") | productos['DIA'].str.contains("vie")]
dias = []
for dia in productos['DIA']:
    if not dia in dias:
        dias.append(dia)
dias

[u'SABADO',
 u'DIA:LUNES ',
 u'DIA LUNES ',
 u'DIA DOMIN',
 u'DIA MARTES ',
 u'MIERCOLES',
 u'DIA MIERCOLES ',
 u'DIA MIERCO',
 u'jueves',
 u'DIA JUEVES  ',
 u'DIA JUEVES ',
 u' DIA VIERNES ',
 u'DIA VIERNES ',
 u'miercoles',
 u'martes',
 u'DIA: SABADO',
 u'DIA: LUNES',
 u'DIA: MARTES',
 u'DIA: MIERCOLES',
 u'DIA: JUEVES',
 u'DIA: VIERNES',
 u'DIA: SABADO 07 07 2012',
 u'DIA: SABADO 07-07-12',
 u'DIA:LUNES  09-07-2012',
 u'DIA: LUNES 9-07-12',
 u'DIA:LUNES      09/07/12',
 u'DIA: LUNES-DOMINGO 08 Y 09 /07/12',
 u'DIA: DOMINGO 08-07-12',
 u'DIA:MARTES 10-07-12',
 u'DIA: MARTES 26-06-12',
 u'DIA: MARTES 10-07-12',
 u'DIA: MIERCOLES 11-07-12',
 u'DIA:MIERCOLES 11/07/12',
 u'DIA:JUEVES        12 /07/12',
 u'DIA: JUEVES 12-07-12',
 u'DIA:VIERNES      13/07/12',
 u'DIA: VIERNES 13-07-12',
 u'DIA: SABADO 14-07-12',
 u'DIA:LUNES  16-07-2012',
 u'DIA: LUNES 16-06-12',
 u'DIA:LUNES  16-07-12',
 u'DIA: LUNES 16-07-12',
 u'DIA: DOMINGO 15-07-12',
 u'DIA:MARTES 17-07-12',
 u'DIA: MARTES 17-07-12',


In [62]:
# Estandarizamos los dias:
dias_real = []
error_indices = []
for dia in dias:
    dia_copy = dia
    error_index = dias.index(dia)
    if "SAB" in dia:
        dia = 'SABADO'
        dias_real.append(dia)
        dia = 'ERROR'
    if "DOM" in dia:
        dia = 'DOMINGO'
        dias_real.append(dia)
        dia = 'ERROR'
    if "LUN" in dia:
        dia = 'LUNES'
        dias_real.append(dia)
        dia = 'ERROR'
    if "MAR" in dia:
        dia = 'MARTES'
        dias_real.append(dia)
        dia = 'ERROR'
    if "MIE" in dia:
        dia = 'MIERCOLES'
        dias_real.append(dia)
        dia = 'ERROR'
    if "JUE" in dia:
        dia = 'JUEVES'
        dias_real.append(dia)
        dia = 'ERROR'
    if "VIE" in dia:
        dia = 'VIERNES'
        dias_real.append(dia)
        dia = 'ERROR'
    if "sab" in dia:
        dia = 'SABADO'
        dias_real.append(dia)
        dia = 'ERROR'
    if "dom" in dia:
        dia = 'DOMINGO'
        dias_real.append(dia)
        dia = 'ERROR'
    if "lun" in dia:
        dia = 'LUNES'
        dias_real.append(dia)
        dia = 'ERROR'
    if "mar" in dia:
        dia = 'MARTES'
        dias_real.append(dia)
        dia = 'ERROR'
    if "mie" in dia:
        dia = 'MIERCOLES'
        dias_real.append(dia)
        dia = 'ERROR'
    if "jue" in dia:
        dia = 'JUEVES'
        dias_real.append(dia)
        dia = 'ERROR'
    if "vie" in dia:
        dia = 'VIERNES'
        dias_real.append(dia)
        dia = 'ERROR'
    if dia_copy == dia:
        error_indices.append(error_index)
        
for error_index in error_indices:
    del dias[error_index]

for i in range(len(dias)):
    productos = productos.replace(dias[i], dias_real[i])

dias = []
for dia in productos['DIA']:
    if not dia in dias:
        dias.append(dia)
dias

['SABADO', 'LUNES', 'DOMINGO', 'MARTES', 'MIERCOLES', 'JUEVES', 'VIERNES']

In [63]:
# Regresamos las observaciones NaN en días
productos = productos.append(productos_dias_nan, ignore_index=True)

In [65]:
productos = productos.loc[productos['SEMANA'] != '']
productos_semanas_nan = productos[productos['SEMANA'].isnull() == True]
productos = productos[productos['SEMANA'].isnull() == False]
semanas = []
for semana in productos['SEMANA']:
    if not semana in semanas:
        semanas.append(semana)
semanas

[u'SEMANA DEL 24 AL 30 DE AGOSTO DEL 2013 ',
 u'ruta  hecha el sabado  se m  santa',
 u'SEMANA DEL 07 AL 13 DE JULIO DEL 2012',
 u'SEMANA DEL 14 AL 20 DE JULIO DEL 2012',
 u'SEMANA DEL 21 AL 27 DE JULIO 2012',
 u'SEMANA DE L21 AL 27 JULIO 2012',
 u'SEMANA DEL  21 AL 27 DE JULIIO 2012',
 u'SEMANA DEL 21 AL 237 DE JULIO 2012',
 u'SAMNA DEL 21 AL 27   DE JULIO 2012',
 u'SAMANA DEL 21 AL 27 DE JULIO 2012',
 u'SEMANA DEL 21 AL 27  DE JULIO 2012',
 u'SEMANA DEL 21 AL 27 2012',
 u'SEMANA DEL 21 AL 27 DE JULIO DE 2102',
 u'SEMANA DEL 28 DE JULIO AL 3 DE AGOSTO 2012',
 u'SEMANA DE L 4 AGOTO AL 10  2012',
 u'SEMANA DEL 4 AL 10 AGOSTO 2012',
 u'SEMANA DEL 4 AL 10 AGOSTO 2012 ',
 u'SEMANA  DEL 4  AL 10 AGOSTO 2012',
 u'SEMANA DEL 4  AL 10 AGOSTO 2012',
 u'SEMANA DEL 4 AL 10 DE AGOSTO 2012',
 u'SEMAN A DEL 4 AL 10 DE AGOSTO 2012',
 u'SEMANA  DE 4  AL 10 AGOSTO 2012',
 u'SEMAN DEL 4 AL 10 AGOSTO 2012',
 u'SEMANA DEL 4 AL 10  AGOSTO 2012',
 u'SEMANA DEL 4 AL 10 AGSTO  2012',
 u'SEMANA DEL 4 A L 10 AG

In [66]:
productos

,PRODUCTO,CARGA,EXISTENCIA,DEVOLUCION,VENTA TOTAL,PRECIO * PRODUC,$,VENTA EN LT,RUTA,DIA,SEMANA,ANIO
4,PRODUCTO10,NaN,NaN,0,0,7.5,0,0,RUTA: SAN MARCOS,LUNES,SEMANA DEL 24 AL 30 DE AGOSTO DEL 2013,2017
5,Queso Hebra,NaN,NaN,0,0,60,0,0,RUTA: SAN MARCOS,LUNES,SEMANA DEL 24 AL 30 DE AGOSTO DEL 2013,2017
6,Queso fresco,NaN,0,0,0,12,0,0,RUTA: SAN MARCOS,LUNES,SEMANA DEL 24 AL 30 DE AGOSTO DEL 2013,2017
7,Queso añejo,NaN,0,0,0,55,0,0,RUTA: SAN MARCOS,LUNES,SEMANA DEL 24 AL 30 DE AGOSTO DEL 2013,2017
8,Queso panela,NaN,0,0,0,50,0,0,RUTA: SAN MARCOS,LUNES,SEMANA DEL 24 AL 30 DE AGOSTO DEL 2013,2017
9,PRODUCTO4,NaN,0,0,0,6,0,0,RUTA: SAN MARCOS,LUNES,SEMANA DEL 24 AL 30 DE AGOSTO DEL 2013,2017
10,PRODUCTO5,NaN,0,0,0,14,0,0,RUTA: SAN MARCOS,LUNES,SEMANA DEL 24 AL 30 DE AGOSTO DEL 2013,2017
11,PRODUCTO1,NaN,0,0,0,9,0,0,RUTA: SAN MARCOS,LUNES,SEMANA DEL 24 AL 30 DE AGOSTO DEL 2013,2017
12,PRODUCTO7,NaN,0,0,0,5,0,0,RUTA: SAN MARCOS,LUNES,SEMANA DEL 24 AL 30 DE AGOSTO DEL 2013,2017
13,PRODUCTO3,NaN,0,0,0,7.5,0,0,RUTA: SAN MARCOS,LUNES,SEMANA DEL 24 AL 30 DE AGOSTO DEL 2013,2017


In [67]:
# Estandarizamos semanas

# Funcion auxiliar: 

def is_int(value):
  try:
    int(value)
    return True
  except ValueError:
    return False

semanas_real = []
error_indices = []
for semana in semanas:
    semana_copy = semana
    error_index = semanas.index(semana)
    if 'ene' in semana or 'ene'.upper() in semana:
        first_digit = semana[re.search("\d", semana).start():re.search("\d", semana).start() + 2]
        if is_int(first_digit[1]):
            first_digit = 1 + int(first_digit)
        else:
            first_digit = 1 + int(first_digit[0])
        semana = str(first_digit / 7 + 1) + ' ENERO' 
        semanas_real.append(semana)
    if 'feb' in semana or 'feb'.upper() in semana:
        first_digit = semana[re.search("\d", semana).start():re.search("\d", semana).start() + 2]
        if is_int(first_digit[1]):
            first_digit = 1 + int(first_digit)
        else:
            first_digit = 1 + int(first_digit[0])
        semana = str(first_digit / 7 + 1) + ' FEBRERO' 
        semanas_real.append(semana)
    if 'mar' in semana or 'mar'.upper() in semana:
        first_digit = semana[re.search("\d", semana).start():re.search("\d", semana).start() + 2]
        if is_int(first_digit[1]):
            first_digit = 1 + int(first_digit)
        else:
            first_digit = 1 + int(first_digit[0])
        semana = str(first_digit / 7 + 1) + ' MARZO' 
        semanas_real.append(semana)
    if 'abr' in semana or 'abr'.upper() in semana:
        first_digit = semana[re.search("\d", semana).start():re.search("\d", semana).start() + 2]
        if is_int(first_digit[1]):
            first_digit = 1 + int(first_digit)
        else:
            first_digit = 1 + int(first_digit[0])
        semana = str(first_digit / 7 + 1) + ' ABRIL' 
        semanas_real.append(semana)
    if 'may' in semana or 'may'.upper() in semana:
        first_digit = semana[re.search("\d", semana).start():re.search("\d", semana).start() + 2]
        if is_int(first_digit[1]):
            first_digit = 1 + int(first_digit)
        else:
            first_digit = 1 + int(first_digit[0])
        semana = str(first_digit / 7 + 1) + ' MAYO' 
        semanas_real.append(semana)
    if 'jun' in semana or 'jun'.upper() in semana:
        first_digit = semana[re.search("\d", semana).start():re.search("\d", semana).start() + 2]
        if is_int(first_digit[1]):
            first_digit = 1 + int(first_digit)
        else:
            first_digit = 1 + int(first_digit[0])
        semana = str(first_digit / 7 + 1) + ' JUNIO' 
        semanas_real.append(semana)
    if 'jul' in semana or 'jul'.upper() in semana:
        first_digit = semana[re.search("\d", semana).start():re.search("\d", semana).start() + 2]
        if is_int(first_digit[1]):
            first_digit = 1 + int(first_digit)
        else:
            first_digit = 1 + int(first_digit[0])
        semana = str(first_digit / 7 + 1) + ' JULIO' 
        semanas_real.append(semana)
    if 'ago' in semana or 'ago'.upper() in semana:
        first_digit = semana[re.search("\d", semana).start():re.search("\d", semana).start() + 2]
        if is_int(first_digit[1]):
            first_digit = 1 + int(first_digit)
        else:
            first_digit = 1 + int(first_digit[0])
        semana = str(first_digit / 7 + 1) + ' AGOSTO' 
        semanas_real.append(semana)
    if 'sep' in semana or 'sep'.upper() in semana:
        first_digit = semana[re.search("\d", semana).start():re.search("\d", semana).start() + 2]
        if is_int(first_digit[1]):
            first_digit = 1 + int(first_digit)
        else:
            first_digit = 1 + int(first_digit[0])
        semana = str(first_digit / 7 + 1) + ' SEPTIEMBRE' 
        semanas_real.append(semana)
    if 'oct' in semana or 'oct'.upper() in semana:
        first_digit = semana[re.search("\d", semana).start():re.search("\d", semana).start() + 2]
        if is_int(first_digit[1]):
            first_digit = 1 + int(first_digit)
        else:
            first_digit = 1 + int(first_digit[0])
        semana = str(first_digit / 7 + 1) + ' OCTUBRE' 
        semanas_real.append(semana)
    if 'nov' in semana or 'nov'.upper() in semana:
        first_digit = semana[re.search("\d", semana).start():re.search("\d", semana).start() + 2]
        if is_int(first_digit[1]):
            first_digit = 1 + int(first_digit)
        else:
            first_digit = 1 + int(first_digit[0])
        semana = str(first_digit / 7 + 1) + ' NOVIEMBRE' 
        semanas_real.append(semana)
    if 'dic' in semana or 'dic'.upper() in semana:
        first_digit = semana[re.search("\d", semana).start():re.search("\d", semana).start() + 2]
        if is_int(first_digit[1]):
            first_digit = 1 + int(first_digit)
        else:
            first_digit = 1 + int(first_digit[0])
        semana = str(first_digit / 7 + 1) + ' DICIEMBRE' 
        semanas_real.append(semana)
    if semana_copy == semana:
        error_indices.append(error_index)
        
for error_index in error_indices:
    del semanas[error_index]

        
for i in range(len(semanas)):
    productos = productos.replace(semanas[i], semanas_real[i])
        
semanas = []
for semana in productos['SEMANA']:
    if not semana in semanas:
        semanas.append(semana)
semanas

['4 AGOSTO',
 u'ruta  hecha el sabado  se m  santa',
 '2 JULIO',
 '3 JULIO',
 '4 JULIO',
 u'SEMANA DEL 21 AL 27 DE JULIO DE 2102',
 '5 JULIO',
 '1 AGOSTO',
 u'SEMANA DEL 4 AL 10 DE AGOSTO 21402',
 '6 AGOSTO',
 '2 AGOSTO',
 '3 AGOSTO',
 '1 SEPTIEMBRE',
 u'SEMANA DEL 1 AL 7 DE SEPTIEMBRE 2012',
 u'SEMA NA DEL 1 AL 7 DE SEPTIMBRE 2012',
 '2 SEPTIEMBRE',
 '3 SEPTIEMBRE',
 '4 SEPTIEMBRE',
 '5 SEPTIEMBRE',
 '5 OCTUBRE',
 '2 OCTUBRE',
 '3 OCTUBRE',
 '4 OCTUBRE',
 '1 OCTUBRE',
 '1 NOVIEMBRE',
 u'SEMANA DEL 3    DE 9   DE NOVIEMBRE   2012',
 '2 NOVIEMBRE',
 '3 NOVIEMBRE',
 '4 NOVIEMBRE',
 u'SEMANA DEL 10     AL 16     DE    NOVIEMBRE  2012',
 u'SEMANA DEL 10     AL 16      DE NOVIEMBRE   2012',
 u'SEMANA DEL 24      AL 30        DE NOVIEMBRE 2012',
 '1 DICIEMBRE',
 '2 DICIEMBRE',
 '3 DICIEMBRE',
 '4 DICIEMBRE',
 '5 ENERO',
 '1 ENERO',
 u'SEMANA DEL    5     AL 11       DE  ENERO  2013',
 u'SEMANA D5          AL    11      DE  ENERO 2013  ',
 '2 ENERO',
 '3 ENERO',
 '4 ENERO',
 '1 FEBRERO',
 '2 

In [68]:
# Para el toque final, quitamos todas las excepciones
productos = productos.loc[productos['SEMANA'].str.contains('1 ENERO')|
							productos['SEMANA'].str.contains('2 ENERO')|
							productos['SEMANA'].str.contains('3 ENERO')|
							productos['SEMANA'].str.contains('4 ENERO')|
							productos['SEMANA'].str.contains('1 FEBRERO')|
							productos['SEMANA'].str.contains('2 FEBRERO')|
							productos['SEMANA'].str.contains('3 FEBRERO')|
							productos['SEMANA'].str.contains('4 FEBRERO')|
							productos['SEMANA'].str.contains('1 MARZO')|
							productos['SEMANA'].str.contains('2 MARZO')|
							productos['SEMANA'].str.contains('3 MARZO')|
							productos['SEMANA'].str.contains('4 MARZO')|
							productos['SEMANA'].str.contains('1 ABRIL')|
							productos['SEMANA'].str.contains('2 ABRIL')|
							productos['SEMANA'].str.contains('3 ABRIL')|
							productos['SEMANA'].str.contains('4 ABRIL')|
							productos['SEMANA'].str.contains('1 MAYO')|
							productos['SEMANA'].str.contains('2 MAYO')|
							productos['SEMANA'].str.contains('3 MAYO')|
							productos['SEMANA'].str.contains('4 MAYO')|
							productos['SEMANA'].str.contains('1 JUNIO')|
							productos['SEMANA'].str.contains('2 JUNIO')|
							productos['SEMANA'].str.contains('3 JUNIO')|
							productos['SEMANA'].str.contains('4 JUNIO')|
							productos['SEMANA'].str.contains('1 JULIO')|
							productos['SEMANA'].str.contains('2 JULIO')|
							productos['SEMANA'].str.contains('3 JULIO')|
							productos['SEMANA'].str.contains('4 JULIO')|
							productos['SEMANA'].str.contains('1 AGOSTO')|
							productos['SEMANA'].str.contains('2 AGOSTO')|
							productos['SEMANA'].str.contains('3 AGOSTO')|
							productos['SEMANA'].str.contains('4 AGOSTO')|
							productos['SEMANA'].str.contains('1 SEPTIEMBRE')|
							productos['SEMANA'].str.contains('2 SEPTIEMBRE')|
							productos['SEMANA'].str.contains('3 SEPTIEMBRE')|
							productos['SEMANA'].str.contains('4 SEPTIEMBRE')|
							productos['SEMANA'].str.contains('1 OCTUBRE')|
							productos['SEMANA'].str.contains('2 OCTUBRE')|
							productos['SEMANA'].str.contains('3 OCTUBRE')|
							productos['SEMANA'].str.contains('4 OCTUBRE')|
							productos['SEMANA'].str.contains('1 NOVIEMBRE')|
							productos['SEMANA'].str.contains('2 NOVIEMBRE')|
							productos['SEMANA'].str.contains('3 NOVIEMBRE')|
							productos['SEMANA'].str.contains('4 NOVIEMBRE')|
							productos['SEMANA'].str.contains('1 DICIEMBRE')|
							productos['SEMANA'].str.contains('2 DICIEMBRE')|
							productos['SEMANA'].str.contains('3 DICIEMBRE')|
							productos['SEMANA'].str.contains('4 DICIEMBRE')]

semanas = []
for semana in productos['SEMANA']:
    if not semana in semanas:
        semanas.append(semana)
semanas

['4 AGOSTO',
 '2 JULIO',
 '3 JULIO',
 '4 JULIO',
 '1 AGOSTO',
 '2 AGOSTO',
 '3 AGOSTO',
 '1 SEPTIEMBRE',
 '2 SEPTIEMBRE',
 '3 SEPTIEMBRE',
 '4 SEPTIEMBRE',
 '2 OCTUBRE',
 '3 OCTUBRE',
 '4 OCTUBRE',
 '1 OCTUBRE',
 '1 NOVIEMBRE',
 '2 NOVIEMBRE',
 '3 NOVIEMBRE',
 '4 NOVIEMBRE',
 '1 DICIEMBRE',
 '2 DICIEMBRE',
 '3 DICIEMBRE',
 '4 DICIEMBRE',
 '1 ENERO',
 '2 ENERO',
 '3 ENERO',
 '4 ENERO',
 '1 FEBRERO',
 '2 FEBRERO',
 '3 FEBRERO',
 '4 FEBRERO',
 '1 MARZO',
 '2 MARZO',
 '3 MARZO',
 '4 MARZO',
 '2 ABRIL',
 '3 ABRIL',
 '4 ABRIL',
 '1 MAYO',
 '2 MAYO',
 '3 MAYO',
 '4 MAYO',
 '1 JUNIO',
 '2 JUNIO',
 '3 JUNIO',
 '4 JUNIO',
 '1 ABRIL',
 '1 JULIO']

In [69]:
# Regresamos las observaciones NaN en semanas
productos = productos.append(productos_semanas_nan, ignore_index=True)

In [70]:
# La base de productos limpia!
productos

,PRODUCTO,CARGA,EXISTENCIA,DEVOLUCION,VENTA TOTAL,PRECIO * PRODUC,$,VENTA EN LT,RUTA,DIA,SEMANA,ANIO
0,PRODUCTO10,NaN,NaN,0,0,7.5,0,0,RUTA: SAN MARCOS,LUNES,4 AGOSTO,2017
1,Queso Hebra,NaN,NaN,0,0,60,0,0,RUTA: SAN MARCOS,LUNES,4 AGOSTO,2017
2,Queso fresco,NaN,0,0,0,12,0,0,RUTA: SAN MARCOS,LUNES,4 AGOSTO,2017
3,Queso añejo,NaN,0,0,0,55,0,0,RUTA: SAN MARCOS,LUNES,4 AGOSTO,2017
4,Queso panela,NaN,0,0,0,50,0,0,RUTA: SAN MARCOS,LUNES,4 AGOSTO,2017
5,PRODUCTO4,NaN,0,0,0,6,0,0,RUTA: SAN MARCOS,LUNES,4 AGOSTO,2017
6,PRODUCTO5,NaN,0,0,0,14,0,0,RUTA: SAN MARCOS,LUNES,4 AGOSTO,2017
7,PRODUCTO1,NaN,0,0,0,9,0,0,RUTA: SAN MARCOS,LUNES,4 AGOSTO,2017
8,PRODUCTO7,NaN,0,0,0,5,0,0,RUTA: SAN MARCOS,LUNES,4 AGOSTO,2017
9,PRODUCTO3,NaN,0,0,0,7.5,0,0,RUTA: SAN MARCOS,LUNES,4 AGOSTO,2017


In [71]:
productos.to_pickle('productos_limpia')